In [1]:
import os
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm

# from utils_subdivision.gen_distribution_single_plots import analyze_phases
# from utils_subdivision.gen_distribution_subplot import analyze_single_type    # plot_combined_results
# from utils_subdivision.gen_distribution_merged_plot import plot_merged
from utils_dot_plot.drum_single import analyze_phases
# from utils_dot_plot.drum_merged import plot_merged_per_mode

from utils_subdivision.gen_distribution_subplot import analyze_single_type

### Generate individual drum dot plot by mode

In [ ]:
m_idx = 0
mode = ["group", "individual", "audience"]
dance_mode = mode[m_idx]

with open('data/selected_piece_list.pkl', 'rb') as f:
    piece_list = pickle.load(f)
    
    
    
figsize1 = (10, 3)  
dpi1 = 200         
use_window = True  # Changed to True since we're using time segments
onset_types = ["Dun", "J1", "J2"]

    
for file_name in piece_list:
    print(file_name)
    cycles_csv_path = f"data/virtual_cycles/{file_name}_C.csv"
    onsets_csv_path = f"data/drum_onsets/{file_name}.csv"
    dmode_path = f"data/dance_modes_ts/{file_name}_{dance_mode}.pkl"
    
    left_onset_path = f"data/logs_v4_0.007_foot_jun3/{file_name}_T/onset_info/{file_name}_T_left_foot_onsets.csv"
    right_onset_path = f"data/logs_v4_0.007_foot_jun3/{file_name}_T/onset_info/{file_name}_T_right_foot_onsets.csv"
    
    left_onsets = pd.read_csv(left_onset_path)["time_sec"].values
    right_onsets = pd.read_csv(right_onset_path)["time_sec"].values
    
        
    if os.path.exists(dmode_path):
        with open(dmode_path, "rb") as f:
            dance_mode_time_segments = pickle.load(f)       # list of tuples (start_time, end_time)
    else:
        continue

    for onset_type in onset_types:
        save_dir = f"output_dot_plots/drum_single/{dance_mode}/{onset_type}"
        os.makedirs(save_dir, exist_ok=True)
        # Update save path to include mode information
        if not use_window:
            save_path = f"output_static_plot\\{file_name}_{onset_type}_full_duration_subplot.png"
        else:
            save_path = os.path.join(save_dir, f"{file_name}_{onset_type}_{mode[m_idx]}.png")
        
        # Analyze phases and save plot with dance_mode_time_segments
        analyze_phases(
            cycles_csv_path, onsets_csv_path, onset_type,
            dance_mode_time_segments=dance_mode_time_segments,dance_mode = dance_mode,
            save_path=save_path, figsize=figsize1, dpi=dpi1,
            use_window=use_window
        )
        
        print(f"Saved plot for {onset_type} to {save_path}")

### Generate merged drum dot plot by mode

In [11]:
def plot_merged_stacked(file_name,
                       cycles_csv_path,
                       onsets_csv_path,
                       dance_mode_time_segments,
                       W_start=None,
                       W_end=None,
                       figsize=(10, 12),
                       dpi=100,
                       use_window=True,
                       ):
    """Create a single plot showing merged analysis for Dun, J1, and J2
    with stacked, colored scatter and combined KDE."""
    
    # 1) define major xticks (0–400 in ~12 segments)
    xtick = [0, 33, 67, 100, 133, 167, 200, 233, 267, 300, 333, 367, 400]

    # 2) set up single merged axes
    fig, ax = plt.subplots(figsize=figsize, dpi=dpi)
    onset_types = ['Dun', 'J1', 'J2']
    colors = ['#1f77b4', '#2ca02c', '#d62728']  # Blue, Green, Red
    
    # vertical stacking ranges for each type
    vertical_ranges = {
        'Dun': (1, 6),    # y from 1–6
        'J1':  (8, 13),   # y from 8–13
        'J2':  (15, 20),  # y from 15–20
    }

    combined_h = None
    combined_xx = None

    # 3) loop onset types, plot colored scatter
    for onset_type, color in zip(onset_types, colors):
        # For each time segment, analyze and combine the results
        all_phases = []
        all_window_pos = []
        segment_kde_h = None
        segment_kde_xx = None

        for start_time, end_time in dance_mode_time_segments:
            phases, window_pos, kde_xx, kde_h = analyze_single_type(
                cycles_csv_path, onsets_csv_path,
                onset_type, start_time, end_time,  # Use segment times
                use_window=True  # Force window mode for segments
            )
            
            if phases is not None:
                all_phases.extend(phases)
                all_window_pos.extend(window_pos)
                
                # Accumulate KDE
                if segment_kde_h is None:
                    segment_kde_h = kde_h.copy()
                    segment_kde_xx = kde_xx.copy()
                else:
                    segment_kde_h += kde_h

        if not all_phases:  # Skip if no data found
            continue

        # Convert lists to numpy arrays
        phases = np.array(all_phases)
        window_pos = np.array(all_window_pos)

        # stack y
        y0, y1 = vertical_ranges[onset_type]
        y_scaled = y0 + (window_pos * (y1 - y0))
            
        # Plot scatter with single color for each instrument
        ax.scatter(phases * 400,
                   y_scaled,
                   s=5, alpha=0.6,
                   color=color,
                   label=onset_type)

        # accumulate KDE
        if combined_h is None:
            combined_h = segment_kde_h.copy()
            combined_xx = segment_kde_xx.copy()
        else:
            combined_h += segment_kde_h

    # 4) draw combined KDE at bottom (-5 to 0)
    if combined_h is not None:
        # normalize to 0–1
        combined_h = combined_h / np.max(combined_h)
        kde_scaled = -5 + (5 * combined_h)
        ax.fill_between(combined_xx * 400,
                        -5, kde_scaled,
                        alpha=0.3, color='purple',
                        label='Combined KDE')

    # 5) Add subdivision lines
    for subdiv in range(1, 13):  # 12 subdivisions
        x_pos = ((subdiv-1) * 400) / 12
        color = get_subdiv_color(subdiv)
        ax.vlines(x_pos, -5.5, 20.5,
                  color=color, linestyle='-', linewidth=1)

    
    # 5) styling & axes
    ax.set_xlim(0, 400)
    ax.set_xticks(xtick)
    ax.set_xlabel('Metric cycle (0-400)')
    
    ax.set_ylim(-5.5, 20.5)
    ax.set_yticks([3, 10, 17])
    ax.set_yticklabels(['Dun', 'J1', 'J2'])
    
    ax.set_ylabel('Instrument')
    ax.grid(True, alpha=0.3)
    ax.set_xlim(-33, 400)

    # 6) title & legend
    title = f'File: {file_name} | Dance Mode: {dance_mode}'
    title += f' | Segments: {len(dance_mode_time_segments)}' if use_window else ' | Full Recording'
    ax.set_title(title, pad=10)
    
    # Add legend
    ax.legend(loc='center right', framealpha=0.5, fontsize=6)

    return fig, ax

def get_subdiv_color(subdiv):
    if subdiv in [1, 4, 7, 10]:
        return 'black'
    elif subdiv in [2, 5, 8, 11]:
        return 'green'
    elif subdiv in [3, 6, 9, 12]:
        return 'red'
    return 'gray'

In [ ]:
# Main execution code
m_idx = 0
mode = ["group", "individual", "audience"]
dance_mode = mode[m_idx]

with open('data/selected_piece_list.pkl', 'rb') as f:
    piece_list = pickle.load(f)
    
for file_name in piece_list:
    print(file_name)
    cycles_csv_path = f"data/virtual_cycles/{file_name}_C.csv"
    onsets_csv_path = f"data/drum_onsets/{file_name}.csv"
    dmode_path = f"data/dance_modes_ts/{file_name}_{dance_mode}.pkl"
    
    if os.path.exists(dmode_path):
        with open(dmode_path, "rb") as f:
            dance_mode_time_segments = pickle.load(f)
            
        # Call the modified function
        fig, ax = plot_merged_stacked(
            file_name=file_name,
            cycles_csv_path=cycles_csv_path,
            onsets_csv_path=onsets_csv_path,
            dance_mode_time_segments=dance_mode_time_segments,
            figsize=(10, 3),
            dpi=200,
            use_window=True
        )
        # plt.show()
        
        save_dir = f"output_dot_plots/drum_merged/{dance_mode}"
        os.makedirs(save_dir, exist_ok=True)
        save_path = os.path.join(save_dir, f"{file_name}_{dance_mode}_merged.png")
        plt.savefig(save_path, bbox_inches='tight')  # Add bbox_inches='tight' to prevent label clipping
        plt.close()


### Generate Dance dot plot by mode

In [25]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
from utils_subdivision.gen_distribution_single_plots import find_cycle_phases, kde_estimate

def get_subdiv_color(subdiv):
    if subdiv in [1, 4, 7, 10]:
        return 'black'
    elif subdiv in [2, 5, 8, 11]:
        return 'green'
    elif subdiv in [3, 6, 9, 12]:
        return 'red'
    return 'gray'

def plot_foot_onsets_stacked(file_name,
                             cycles_csv_path,
                             left_onsets,
                             right_onsets,
                             dance_mode_time_segments,
                             figsize=(10, 3),
                             dpi=200,
                             use_window=True):
    """Plot left and right foot onsets with stacked scatter and combined KDE, using robust phase and KDE calculation."""
    
    xtick = [0, 33, 67, 100, 133, 167, 200, 233, 267, 300, 333, 367, 400]
    fig, ax = plt.subplots(figsize=figsize, dpi=dpi)
    cycles = pd.read_csv(cycles_csv_path)["Virtual Onset"].values

    vertical_ranges = {
        'left': (1, 6),
        'right': (8, 13),                    # 'right': (1, 6),       # 'right': (8, 13)
    }

    combined_phases = []

    for foot_type, onsets, color in [('left', left_onsets, '#1f77b4'), ('right', right_onsets, '#d62728')]:
        # Filter onsets by time segments
        if use_window:
            window_mask = np.zeros(len(onsets), dtype=bool)
            for W_start, W_end in dance_mode_time_segments:
                segment_mask = (onsets >= W_start) & (onsets <= W_end)
                window_mask |= segment_mask
            filtered_onsets = onsets[window_mask]
        else:
            filtered_onsets = onsets

        if len(filtered_onsets) == 0:
            continue

        # Use robust phase calculation
        cycle_indices, phases, valid_onsets = find_cycle_phases(filtered_onsets, cycles)
        if len(phases) == 0:
            continue

        # Collect for combined KDE
        combined_phases.extend(phases)

        # Calculate window positions
        window_positions = []
        if use_window:
            for onset in valid_onsets:
                for seg_idx, (W_start, W_end) in enumerate(dance_mode_time_segments):
                    if W_start <= onset <= W_end:
                        segment_duration = W_end - W_start
                        relative_pos = (onset - W_start) / segment_duration
                        window_pos = seg_idx + relative_pos
                        window_positions.append(window_pos)
                        break
        else:
            window_positions = np.zeros_like(valid_onsets)

        window_positions = np.array(window_positions)
        y0, y1 = vertical_ranges[foot_type]
        y_scaled = y0 + (window_positions * (y1 - y0))

        # Plot scatter
        ax.scatter(phases * 400, y_scaled, s=5, alpha=0.6, color=color, label=f'{foot_type.capitalize()} Foot')

    # Combined KDE at bottom using kde_estimate
    if len(combined_phases) > 0:
        kde_xx, kde_h = kde_estimate(np.array(combined_phases), SIG=0.01)
        # Only plot the region that maps to the x-axis
        mask = (kde_xx * 400 >= -33) & (kde_xx * 400 <= 400)
        kde_xx_plot = kde_xx[mask]
        kde_h_plot = kde_h[mask]
        if np.max(kde_h_plot) > 0:
            kde_scaled = -5 + (5 * kde_h_plot / np.max(kde_h_plot))
            ax.fill_between(kde_xx_plot * 400, -5, kde_scaled, alpha=0.3, color='purple', label='Combined KDE')

    # Subdivision lines
    for i in range(1, 13):
        x_pos = ((i-1) * 400) / 12
        color = get_subdiv_color(i)
        ax.vlines(x_pos, -5.5, 13.5, color=color, linestyle='-', linewidth=1)

    # Styling
    ax.set_xticks(xtick)
    ax.set_xlabel('Metric cycle (0-400)')
    ax.set_ylim(-5.5, 13.5)
    ax.set_yticks([3, 10])
    ax.set_yticklabels(['Left Foot', 'Right Foot'])
    ax.set_ylabel('Foot')
    ax.grid(True, alpha=0.3)
    ax.set_xlim(-33, 400)

    # Title & legend
    title = f'File: {file_name} | Dance Mode: {dance_mode}'
    title += f' | Segments: {len(dance_mode_time_segments)}' if use_window else ' | Full Recording'
    ax.set_title(title, pad=10)
    ax.legend(loc='center right', framealpha=0.5, fontsize=6)

    return fig, ax



In [ ]:
# Example usage (main execution code)
m_idx = 0
mode = ["group", "individual", "audience"]
dance_mode = mode[m_idx]

with open('data/selected_piece_list.pkl', 'rb') as f:
    piece_list = pickle.load(f)
    
for file_name in piece_list:
    print(file_name)
    cycles_csv_path = f"data/virtual_cycles/{file_name}_C.csv"
    dmode_path = f"data/dance_modes_ts/{file_name}_{dance_mode}.pkl"
    
    left_onset_path = f"data/logs_v4_0.007_foot_jun3/{file_name}_T/onset_info/{file_name}_T_left_foot_onsets.csv"
    right_onset_path = f"data/logs_v4_0.007_foot_jun3/{file_name}_T/onset_info/{file_name}_T_right_foot_onsets.csv"
    
    left_onsets = pd.read_csv(left_onset_path)["time_sec"].values
    right_onsets = pd.read_csv(right_onset_path)["time_sec"].values
    
    if os.path.exists(dmode_path):
        with open(dmode_path, "rb") as f:
            dance_mode_time_segments = pickle.load(f)
            
        fig, ax = plot_foot_onsets_stacked(
            file_name=file_name,
            cycles_csv_path=cycles_csv_path,
            left_onsets=left_onsets,
            right_onsets=right_onsets,
            dance_mode_time_segments=dance_mode_time_segments,
            figsize=(10, 3),
            dpi=200,
            use_window=True
        )
        # plt.show()
        save_dir = f"output_dot_plots/dance/{dance_mode}"
        os.makedirs(save_dir, exist_ok=True)
        save_path = os.path.join(save_dir, f"{file_name}_{dance_mode}_merged.png")
        plt.savefig(save_path, bbox_inches='tight')  # Add bbox_inches='tight' to prevent label clipping
        plt.close()
    # break

### Combine PNG vertically

In [ ]:
import os
from PIL import Image

dance_plots = "output_dot_plots/dance/audience"
drum_plots = "output_dot_plots/drum_merged/audience"
save_dir = "output_dot_plots/combined"

os.makedirs(save_dir, exist_ok=True)

# Get the intersection of filenames in both directories
dance_files = set(os.listdir(dance_plots))
drum_files = set(os.listdir(drum_plots))
common_files = dance_files & drum_files

for fname in common_files:
    dance_img_path = os.path.join(dance_plots, fname)
    drum_img_path = os.path.join(drum_plots, fname)
    save_path = os.path.join(save_dir, fname)

    # Open images
    img1 = Image.open(dance_img_path)
    img2 = Image.open(drum_img_path)

    # Make sure widths match (optional: resize if needed)
    if img1.width != img2.width:
        # Resize img2 to match img1 width, keeping aspect ratio
        new_height = int(img2.height * img1.width / img2.width)
        img2 = img2.resize((img1.width, new_height), Image.Resampling.LANCZOS)

    # Create new image with combined height
    total_height = img1.height + img2.height
    combined_img = Image.new('RGB', (img1.width, total_height), (255, 255, 255))
    combined_img.paste(img1, (0, 0))
    combined_img.paste(img2, (0, img1.height))

    # Save
    combined_img.save(save_path)
    print(f"Saved: {save_path}")